In [2]:
# 约瑟夫环问题
# 假设有n个人， 从第一个人开始报数， 数到m的人被淘汰

# person = ['Bob', 'Jack', 'Allen', 'Tony', 'Peter', 'Rose', 'Wade']

# class Person:
#     def __init__(self, name, id):
#         self.name = name
#         self.id = id
            
#     def print_info(self):
#         print("id:{}, name:{}" .format(self.id, self.name))

# p1 = Person('Bob', 1)
# p2 = Person('Jack', 2)
# p3 = Person('Allen', 3)
# p4 = Person('Tony', 4)
# p5 = Person('Peter', 5)
# p6 = Person('Rose', 6)
# p7 = Person('Wade', 7)
# participants = [p1, p2, p3, p4, p5, p6, p7]

class Josephus:

    class Person:
        def __init__(self, name, id):
            self.name = name
            self.id = id
        
        def print_info(self):
            print("id:{}, name:{}" .format(self.id, self.name))

    def __init__(self, participants, step, start):
        self.participants = participants
        self.step = step
        self.start = start

    def josephus(self):
        n = len(self.participants)
        # 将所有人的序号按顺序（1 ~ n）生成一个列表
        sequence = list(range(1, n+1))
        index = self.start - 1
        result = []
        for i in range(n):
            count = 0
            while count < self.step:
                if sequence[index] != 0:
                    count += 1
                if count == self.step:
                    result.append(self.participants[index])
                    sequence[index] = 0
                index = (index + 1) % n
        return result   

    def print_result(self, reslut):
        for i in range(len(reslut) - 1):
            print("第%d位被淘汰" %(i+1))   
            reslut[i].print_info()
        print("获胜者：")
        reslut[len(reslut) - 1].print_info()

p1 = Josephus.Person('Bob', 1)
p2 = Josephus.Person('Jack', 2)
p3 = Josephus.Person('Allen', 3)
p4 = Josephus.Person('Tony', 4)
p5 = Josephus.Person('Peter', 5)
p6 = Josephus.Person('Rose', 6)
p7 = Josephus.Person('Wade', 7)
participants = [p1, p2, p3, p4, p5, p6, p7]


# assert(josephus([0, 1, 2, 3], 3) == [2, 1, 3, 0])
jos = Josephus(participants, 4, 3)

rank = jos.josephus()
jos.print_result(rank)

第1位被淘汰
id:6, name:Rose
第2位被淘汰
id:3, name:Allen
第3位被淘汰
id:1, name:Bob
第4位被淘汰
id:7, name:Wade
第5位被淘汰
id:2, name:Jack
第6位被淘汰
id:5, name:Peter
获胜者：
id:4, name:Tony
